In [2]:
import gymnasium as gym
import ventilation_env 
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env

pygame 2.5.1 (SDL 2.28.2, Python 3.11.0)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [1]:
import cv2

In [10]:
vent_env = gym.make("Ventilation")
vect_env = make_vec_env(vent_env)

TypeError: 'TimeLimit' object is not callable

In [9]:
vent_env = gym.make("Ventilation")
vect_env = make_vec_env(vent_env)
vent_model = PPO("MlpPolicy", vent_env, verbose=1)
vent_model.learn(total_timesteps=25000)
vent_model.save("ppo_ventilation")

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


TypeError: 'NoneType' object is not iterable

In [2]:
from stable_baselines3.common.torch_layers import BaseFeaturesExtractor
import torch
from torch import nn
import gymnasium as gym

class MinigridFeaturesExtractor(BaseFeaturesExtractor):
    def __init__(self, observation_space: gym.Space, features_dim: int = 512, normalized_image: bool = False) -> None:
        super().__init__(observation_space, features_dim)
        n_input_channels = observation_space.shape[0]
        self.cnn = nn.Sequential(
            nn.Conv2d(n_input_channels, 16, (2, 2)),
            nn.ReLU(),
            nn.Conv2d(16, 32, (2, 2)),
            nn.ReLU(),
            nn.Conv2d(32, 64, (2, 2)),
            nn.ReLU(),
            nn.Flatten(),
        )

        # Compute shape by doing one forward pass
        with torch.no_grad():
            n_flatten = self.cnn(torch.as_tensor(observation_space.sample()[None]).float()).shape[1]

        self.linear = nn.Sequential(nn.Linear(n_flatten, features_dim), nn.ReLU())

    def forward(self, observations: torch.Tensor) -> torch.Tensor:
        return self.linear(self.cnn(observations))

In [4]:
import minigrid
from minigrid.wrappers import ImgObsWrapper
from stable_baselines3 import PPO
import ventilation_env 


policy_kwargs = dict(
    features_extractor_class=MinigridFeaturesExtractor,
    features_extractor_kwargs=dict(features_dim=128),
)

env = gym.make("Ventilation", render_mode="rgb_array")
env = ImgObsWrapper(env)

model = PPO("CnnPolicy", env, policy_kwargs=policy_kwargs, verbose=1)
model.learn(2e4)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 600      |
|    ep_rew_mean     | 0        |
| time/              |          |
|    fps             | 1279     |
|    iterations      | 1        |
|    time_elapsed    | 1        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 600         |
|    ep_rew_mean          | 0           |
| time/                   |             |
|    fps                  | 798         |
|    iterations           | 2           |
|    time_elapsed         | 5           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.010315164 |
|    clip_fraction        | 0.0524      |
|    clip_range      

In [5]:
model.save("vent_cartpole")

In [8]:
obs = env.reset()

while True:
    action, _states = model.predict(obs)
    obs, rewards, dones, info = env.step(action)
    vec_env.render("human")

AttributeError: 'tuple' object has no attribute 'shape'

In [13]:
from gymnasium.core import ObservationWrapper
from gymnasium.spaces import Box, Dict
import numpy as np
class ObjObsWrapper(ObservationWrapper):
    def __init__(self, env):
        """A wrapper that makes image the only observation.
        Args:
            env: The environment to apply the wrapper
        """
        super().__init__(env)

        self.observation_space = Dict(
            {
                "image": env.observation_space.spaces["image"],
                "mission": Box(low=0.0, high=1.0, shape=(9,), dtype=np.float32),
            }
        )

        self.color_one_hot_dict = {
            "red": np.array([1.0, 0.0, 0.0, 0.0, 0.0, 0.0]),
            "green": np.array([0.0, 1.0, 0.0, 0.0, 0.0, 0.0]),
            "blue": np.array([0.0, 0.0, 1.0, 0.0, 0.0, 0.0]),
            "purple": np.array([0.0, 0.0, 0.0, 1.0, 0.0, 0.0]),
            "yellow": np.array([0.0, 0.0, 0.0, 0.0, 1.0, 0.0]),
            "grey": np.array([0.0, 0.0, 0.0, 0.0, 0.0, 1.0]),
        }

        self.obj_one_hot_dict = {
            "ball": np.array([1.0, 0.0, 0.0]),
            "box": np.array([0.0, 1.0, 0.0]),
            "key": np.array([0.0, 0.0, 1.0]),
        }

    def observation(self, obs):
        mission_array = np.concatenate(
            [
                self.color_one_hot_dict[self.target_color],
                self.obj_one_hot_dict[self.target_obj],
            ]
        )

        wrapped_obs = {
            "image": obs["image"],
            "mission": mission_array,
        }

        return wrapped_obs

In [14]:
env = gym.make("Ventilation", render_mode="human")
env = ObjObsWrapper(env)

ppo = PPO("MultiInputPolicy", env, verbose=1)
ppo = ppo.load("vent_cartpole.zip", env=env)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.


RuntimeError: Calculated padded input size per channel: (7 x 7). Kernel size: (8 x 8). Kernel size can't be greater than actual input size

In [ ]:
# Parallel environments
vec_env = make_vec_env("CartPole-v1", n_envs=4)

model = PPO("MlpPolicy", vec_env, verbose=1)
model.learn(total_timesteps=25000)
model.save("ppo_cartpole")

del model # remove to demonstrate saving and loading

model = PPO.load("ppo_cartpole")

obs = vec_env.reset()
#while True:
##    action, _states = model.predict(obs)
    obs, rewards, dones, info = vec_env.step(action)
    vec_env.render("human")

Using cpu device
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 22.2     |
|    ep_rew_mean     | 22.2     |
| time/              |          |
|    fps             | 7745     |
|    iterations      | 1        |
|    time_elapsed    | 1        |
|    total_timesteps | 8192     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 32.8        |
|    ep_rew_mean          | 32.8        |
| time/                   |             |
|    fps                  | 3259        |
|    iterations           | 2           |
|    time_elapsed         | 5           |
|    total_timesteps      | 16384       |
| train/                  |             |
|    approx_kl            | 0.014580395 |
|    clip_fraction        | 0.208       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.681      |
|    explained_variance   | 0.000206    |
|    learning

In [ ]:
# Parallel environments
vec_env = make_vec_env("CartPole-v1", n_envs=4)

model = PPO("MlpPolicy", vec_env, verbose=1)
model.learn(total_timesteps=25000)
model.save("ppo_cartpole")

del model # remove to demonstrate saving and loading

model = PPO.load("ppo_cartpole")

obs = vec_env.reset()
#while True:
##    action, _states = model.predict(obs)
    obs, rewards, dones, info = vec_env.step(action)
    vec_env.render("human")

Using cpu device
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 22.2     |
|    ep_rew_mean     | 22.2     |
| time/              |          |
|    fps             | 7745     |
|    iterations      | 1        |
|    time_elapsed    | 1        |
|    total_timesteps | 8192     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 32.8        |
|    ep_rew_mean          | 32.8        |
| time/                   |             |
|    fps                  | 3259        |
|    iterations           | 2           |
|    time_elapsed         | 5           |
|    total_timesteps      | 16384       |
| train/                  |             |
|    approx_kl            | 0.014580395 |
|    clip_fraction        | 0.208       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.681      |
|    explained_variance   | 0.000206    |
|    learning